In [1]:
#downloading libraries
import pandas as pd 
import numpy as np
import matplotlib as plt
import pandas as pd
import nltk
import re
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt
from collections import Counter
import pickle

#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Abhishek's
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Loading the pretrained model 

In [2]:
filename = 'rfr_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [3]:
#loading up reddit api in order to extract data from a reddit url
import praw
import pprint
reddit = praw.Reddit(client_id='300HIOocldVcKA', client_secret='PCktLUhpPaRB1RrBCouEDPdpEBU', user_agent='abhishek chopra') # potentially needs configuring, see docs

## Preprocessing Data

* These functions will be used to preprocess raw data before using our loaded on it 
    * preProcessData-> Takes in textual data , tokenizes it and removes stopwords and special characters
    * text_extractor-> Extracts text from a specific column of a DataFrame
    * joiner-> Joins textual tokenized data to form lists of strings



In [4]:
def preProcessData(dataa):
    stopwords_en = list(set(stopwords.words('english')))
    def split(word): 
        return [char for char in word]   
    punchList = split(punctuation)

    print(stopwords_en)
    print('Punctuation :', punchList)

    wordTokenList = [word_tokenize(sent) for sent in dataa]
    lowercasingList = [[word.lower() for word in sentence] for sentence in wordTokenList]
    noStopWordList = [[word for word in sentence if word not in stopwords_en] for sentence in lowercasingList]
    noPunchList = [[re.sub(r'([^\s\w]|_)+', '', word) for word in sentence] for sentence in noStopWordList]
    #noPunchList = [[word for word in sentence if word not in punchList] for sentence in noStopWordList]
    PP_data = [[word for word in sentence if word] for sentence in noPunchList]
    return PP_data

def text_extractor(text,text_type):
    title_list=[]
    for i in range(len(text)):
        title_list.append(text[text_type][i])
    return title_list
def joiner(data):
    input_corrected = [" ".join(i) for i in data]
    return input_corrected

## Detecting Flairs
 
* Using this function I extract data from a reddit post given its URL and then process that data to predict a flair given a loaded ML model


In [5]:
def detect_flair(url,loaded_model):

    submission = reddit.submission(url=url)
    topics_dict = {"title":[], "comments":[]}
    topics_dict["title"].append(submission.title)
    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
    topics_dict["comments"].append(comment)
    
    topics_data = pd.DataFrame(topics_dict)
    feature_combine = topics_data["title"] + topics_data["comments"]
    topics_data = topics_data.assign(feature_combine = feature_combine)
    feature=text_extractor(topics_data,'feature_combine')
    x=joiner(preProcessData(feature))
    return loaded_model.predict(x)

In [6]:
detect_flair('https://www.reddit.com/r/india/comments/cyj7gz/kashmir_news_coverage_3/https://github.com/akshaybhatia10/Reddit-Flair-Detection.git',loaded_model)

['below', 'itself', 'too', 'ours', 'such', "hadn't", 'i', 'has', 'an', 'on', 'all', 'they', 'am', 'or', 'aren', 'of', 'during', 'wouldn', 'those', 'herself', 'she', 'with', 'll', 'o', 'myself', 'can', "that'll", 'do', 'here', 'y', 'what', 'did', 'shan', 'in', 'his', 'from', 'before', 'their', 'after', 'being', 'ain', "needn't", 'how', 'my', 'didn', 'against', 'needn', 'out', "mustn't", 'doing', 'hasn', 'her', 'no', "didn't", 're', 'ma', 'some', 'shouldn', 'same', 'is', 'at', "shan't", 'now', "isn't", 'through', 'further', "you'd", 'our', 'them', 'couldn', 'isn', 'yourself', 'you', 'there', 'under', 'few', 'above', 'weren', 'own', 'other', 'over', 'off', 'him', "it's", 'theirs', 'each', 'when', 'mustn', "won't", 'hers', 'were', 'don', 'which', 'again', 'most', 'himself', 'why', "doesn't", 'then', 'than', 'for', "you'll", 'and', 'up', 'more', 'between', "shouldn't", 'its', 'so', 's', "aren't", "you're", 'having', 'while', "couldn't", 'haven', 'this', 'to', 'we', 'where', 'these', "haven'

array(['Politics'], dtype=object)